<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [2]:
import numpy as np

In [5]:
# 1. Add backprop
# 2. Add the bias term
# 3. Check np.amax() for getting 1s and 0s at the end

class MultiPerceptron:
    def __init__(self):
        # Architecture
        self.inputs = 3
        self.hidden_nodes = 4
        self.output_nodes = 1
        
        # Initial weights
        self.weights1 = np.random.randn(self.inputs, self.hidden_nodes)
        self.weights2 = np.random.randn(self.hidden_nodes, self.output_nodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoid_prime(self, s):
        return s * (1-s)
    
    def feed_forward(self, X):
        # Calculate weighted sum of input layer to first hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of hidden_sums
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted sum between hidden and output layer
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output

In [6]:
X = np.array(([0,0,1],
              [0,1,1],
              [1,0,1],
              [0,1,0],
              [1,0,0],
              [1,1,1],
              [0,0,0]), dtype=float)
y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]), dtype=float)

In [8]:
nn = MultiPerceptron()

print(X[0])
output = nn.feed_forward(X)
print("\nOutput\n", output)

[0. 0. 1.]

Output
 [[0.37564815]
 [0.37485948]
 [0.28914662]
 [0.39513921]
 [0.27993734]
 [0.26443214]
 [0.39069588]]


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [ ]:
##### Your Code Here #####

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?